# Amortised Bayesian Inference in Hierarhical Dynamical Systems

In this notebook, we examine the embedding of a stochastic autoencoder with a reparametrized variational inference distribution -- a variational autoencoder -- within the settings of dynamical systems for efficient amortised inference of parameters and unobserved states. Initially, we exploit a generative process to capture the uncertainty within the model and, concomitantly, instantiate an inferential procedure to delineate the aleatory uncertainty within the measurements. Subsequently, we endow the inference distribution with a mean-field approximation and further design a neural network to estimate its parameters, which amortizes the computations through the sharing of statistical strength between the instances in the data set. 

We start, however, with an exposition of the model's architecture; a reasonably self-contained approach is contained in the [slides](./slides). With this objective, notice that, in a hierarchical dynamical system, there are multiple species interacting with distinct environments; these environments, moreover, are stratifiable in distinct (and, as we will verify, compositional) groups, which, then, characterize a population. Thus, each evironments enforms a parametrizable dynamical system; nevertheless, the parameters may, or may not, be shared within the same group or between distinct population. Henceforth, we will identify its parameters as *species*, whereas its distinct instantiations in different branchs of the hierarchy will be characterized as *components*; equivalently, a component is an instance of a species in a prespecified environment. 

Crucially, in a Bayesian framework, the postulation of a parameter space is equivalent to the identification of a latent space in which the data is properly, although lossy, encoded; hence, we write henceforth $\mathbf{Z}$ for the model's parameters, which are compartimentalized as 

\begin{equation*} 
(\mathbf{z}_{I}, \mathbf{z}_{G}, \mathbf{z}_{P}) \in \mathbf{Z}, 
\end{equation*} 

for the individual, $\mathbf{z}_{I}$, collective, $\mathbf{z}_{G}$, and populational $\mathbf{z}_{P}$, attributes. In these settings, we generative process is as follows. In an initial moment, a latent variable is sampled from an (unknown) distribution, inducing particular strucutral boundaries for the dynamical system, whose realization is subjected to an observer and a noise process, culminating in the current observation. Explicitly, the succeeding algorithm identifies our generative process. 

+ $\mathbf{z} \sim \pi(\mathbf{z})$, 
+ $\dot{\mathbf{x}} = \mathbf{f}(\mathbf{x}, \mathbf{z})$, 
+ $\mathbf{X} = \textrm{Simulate}(\mathbf{f}, \mathbf{X}_{o})$ ($\mathbf{X}_{o}$ equals the initial state of the system), 
+ $M = \psi(\mathbf{X})$, the observer process, $\Sigma = \rho(\mathbf{X}, \mathbf{z})$, the noise process,  
+ $\mathbf{Y} \sim \pi(\mathbf{Y} | M, \Sigma)$; this comprises the observed data. 

Importantly, this model accounts concomitantly for white-box, in which a reasonable dynamical system $\mathbf{f}$ is prespecified by a human, and black-box, with the indexation of the model by a finite-dimensional parameter $\theta$, $\mathbf{f}_{\theta}$, and the succeeding inference on $\theta$ (jointly with $\mathbf{z}$), typically with a neural network. The noisy mappings, $\psi $ and $\rho$, in contrast, capture the partial observability constraints for this endeavor. Thus, this model is designed for inference on both *highly nonlinear* and *partially observable* dynamical systems, bestowing its amenability to multiple circumstances. 

Moreover, the generative process induces, by Bayes' rule, a posterior distribution on $\mathbf{Z}$; precisely, 

\begin{equation*} 
\pi(\mathbf{Z} | \mathbf{Y}) \propto f(\mathbf{Y} | \mathbf{Z}) \pi(\mathbf{Z}), 
\end{equation*} 

in which $f(\mathbf{Y} | \mathbf{Z})$ corresponds to the likelihood function underlying the mapping from the latent space, $\mathbf{Z}$, to the samples, $\mathbf{X}$. Nonetheless, this posterior distribution is in general analytically and computationally intractable; thus, we resort to the variational inference framework through a mean-field approximation and regard the inference as an optimization of a divergence measure between the variational approximation and the model's posterior distribution. More explicitly, we let 

\begin{equation*} 
\mathcal{Q} = \{q \colon \mathbf{Z} \rightarrow \mathbb{R}_{+} | q(\mathbf{Z}) = q_{\psi_{I}} (\mathbf{z}_{I}) q_{\psi_{G}} (\mathbf{z}_{G}) q_{\psi_{P}} (\mathbf{z}_{P})\}, 
\end{equation*}

with $\psi_{I}$, $\psi_{G}$ and $\psi_{P}$ as the variational parameters; nextly, $\psi = (\psi_{I}, \psi_{G}, \psi_{P})$. Then, we factor the data marginal log-likelihood as 

\begin{equation*} 
\log p(\mathbf{Y}) = KL(q(\cdot) || \pi( \cdot | \mathbf{Y})) + ELBO (q(\cdot), \pi(\cdot | \mathbf{Y}))
\end{equation*} 
 
(we write $KL$ for the Kullback-Leibler divergence; $ELBO$, in contrast, equals the *evidence lower bound*) and subsequently tackle the optimization 

\begin{equation*} 
(\hat{\psi_{I}}, \hat{\psi_{G}}, \hat{\psi_{P}}) = \textrm{argmin} \ ELBO(q(\cdot) | \pi(\cdot | \mathbf{Y})).  
\end{equation*} 

Circumstantially, we emphasize that, although we could compute, for each instance $\mathbf{Y}$ in the data set, an optimal parameter $\psi(\mathbf{Y})$ to attain the tighest evidence lower bound, this approach would not scale to massive data settings. Hence, we train a neural network [1] that estimates, for each observation, the optimal variational parameters; this approach, in which we share the data's statistical strength through a parametrized estimator, is called *amortized inference*. Particularly, the coupling between a reparametrized variational distribution with a stochastic variational inference characterizes the *variational autoencoders* framework. 

[1] The network's architecture is intricate, and its design is not crucial for this notebook's purpose; we do notice that the authors use a importance-weighted autoencoder with a doubly reparemetrized gradient estimators to unbiasedly estimate the parameters' gradient for the network's optimizer. 

# Clone the repository

In [3]:
%load_ext autoreload
import os 
import sys 
from IPython.display import display, clear_output 
module = "hds"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
if not os.path.exists(module):
    !git clone https://github.com/tiagodsilva/$module 
    !mv $module/data . 

In [5]:
# Install packages; alternativaly, install the Conda's environment
cmd = f"pip install -r {module}/requirements.txt"
status = os.system(cmd) 
if status == 0: 
    clear_output() 

In [6]:
%autoreload 
import numpy as np 
import torch 
import yaml 
sys.path.append(module) 
from vihds.config import Config, Trainer 
from vihds.datasets import build_datasets 
from vihds.parameters import Parameters 
from vihds.training import Training 
from vihds.xval import XvalMerge 
from vihds.vae import build_model 

In [7]:
# Users' parameters 
specs = f"{module}/specs/dr_constant_icml.yaml" 
experiment = "EXAMPLE" 
seed = 42             # The RNG seed
epochs = 1            # Training epochs
test_epoch = 20       # Test epochs
plot_epoch = 100      # Plot epochs
train_samples = 20    # Training samples (per datapoint)
test_samples = 20     # Testing samples (per datapoint) 
dreg = True           # DReG estimator 
precision_hidden_layers = None      # Quantity of hidden layers
use_gpu = torch.cuda.is_available() # Use GPU 
folds = 4                           # Cross-validation folds 
# Attributes to evauate the models' accuracy 
heldout = None         # A heldout device (as R33S32_Y81C76) 
split = 1              # Split for cross-validation in heldout 
figures = True         # Whether to draw figures 
verbose = True         # Increase the volume in stdout 

In [9]:
# Display the models' specifications
model_specs = yaml.load(open(specs, "r"), 
                    Loader=yaml.loader.SafeLoader) 
print(yaml.dump(model_specs)) 

data:
  conditions:
  - C6
  - C12
  default_devices:
    aR: 0
    aS: 0
  devices:
  - Pcat_Y81C76
  - RS100S32_Y81C76
  - RS100S34_Y81C76
  - R33S32_Y81C76
  - R33S34_Y81C76
  - R33S175_Y81C76
  files:
  - proc140916.csv
  - proc140930.csv
  - proc141006.csv
  - proc141021.csv
  - proc141023.csv
  - proc141028.csv
  groups:
    aR:
    - 0
    - 1
    - 1
    - 2
    - 2
    - 2
    aS:
    - 0
    - 1
    - 2
    - 1
    - 2
    - 3
  pretty_devices:
  - Pcat-Pcat
  - R100-S32
  - R100-S34
  - R33-S32
  - R33-S34
  - R33-S175
  separate_conditions: true
  signals:
  - OD
  - mRFP1
  - EYFP
  - ECFP
model: dr_constant
params:
  constant:
    init_cfp: 0.0
    init_lasR: 0.0
    init_luxR: 0.0
    init_rfp: 0.0
    init_x: 0.002
    init_yfp: 0.0
  global:
    KGR_76:
      distribution: LogNormal
      mu: 2.0
      sigma: 3.0
    KGR_81:
      distribution: LogNormal
      mu: -2.0
      sigma: 3.0
    KGS_76:
      distribution: LogNormal
      mu: -2.0
      sigma: 3.0
    KGS_81

In [10]:
class Namespace(object): 
    def __init__(self, **kwargs): 
        for key, val in kwargs.items(): 
            self.__dict__[key] = val 

In [11]:
args = Namespace( 
    yaml=specs, 
    experiment=experiment,
    seed=seed, 
    epochs=epochs, 
    test_epoch=test_epoch, 
    plot_epoch=plot_epoch, 
    train_samples=train_samples,
    test_samples=test_samples,
    dreg=dreg,
    precision_hidden_layers=precision_hidden_layers,
    gpu=torch.cuda.current_device() if use_gpu else "cpu", 
    folds=folds, 
    heldout=heldout, 
    split=split, 
    figures=figures, 
    verbose=verbose
) 

In [12]:
def run_on_split(args: Namespace, 
                 settings: Namespace, 
                 split: bool=None): 
    """
    Execute a training-testing split. 
    """
    data = build_datasets(args, settings) 
    parameters = Parameters(settings.params) 
    model = build_model(args, settings, data, parameters) 
    training = Training(args, settings, data, parameters, model) 
    # Return the data and the evaluations for the current epoch
    return data, training.run() 

In [13]:
def train(args: Namespace): 
    settings = Config(args) 
    settings.trainer = Trainer(args, add_timestamp=True) 
    data_pair, val_results = run_on_split(args, settings) 
    
    if (val_results is not None) and settings.trainer is not None: 
        xval_merge = XvalMerge(args, settings) 
        xval_merge.add(1, data_pair, val_results) 
        xval_merge.finalize() 
        xval_merge.save() 
        xval_merge.mark_completed(args.experiment) 
        if args.figures: 
            xval_merge.make_writer() 
            xval_merge.make_images() 
            xval_merge.close_writer() 

In [122]:
train(args) 

Processing command-line arguments
- <__main__.Namespace object at 0x7f9e3d8d68e0>
- Setting test_epoch to 1
- Setting plot_epoch to 1
- Setting: np.random.seed(42)
- Setting: torch.manual_seed(42)
- GPU mode computation


hds/vihds/datasets.py:137: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  times_arr = np.asarray(times_list)
hds/vihds/datasets.py:138: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  obs_arr = np.asarray(observations_list)


Initialising encoder
Initialising decoder
---------------------------
Training: split 1 of 4


  0%| | 0/

epoch    1 | train (iwae-elbo = -7252.9907, time = 3.17, total = 3.17)

...

 | val (iwae-elbo = -7554.3672, time = 5.97, total = 5.97)
Preparing cross-validation results
Saving results to results/EXAMPLE_20220813T222914592234
Saving to: results/EXAMPLE_20220813T222914592234
Making summary figure



/home/tiago/anaconda3/envs/viewer/envs/vihds/lib/python3.8/site-packages/numpy/lib/npyio.py:501: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Making treatment figure


hds/vihds/plotting.py:255: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "kx" (-> color='k'). The keyword argument will take precedence.
  ax.semilogx(


Making species figure
Making global parameters figure
-  ['prec_x', 'prec_rfp', 'prec_yfp', 'prec_cfp', 'e76', 'e81', 'KGR_76', 'KGR_81', 'KGS_76', 'KGS_81', 'KR6', 'KR12', 'KS6', 'KS12', 'nR', 'nS', 'aYFP', 'aCFP', 'dR', 'dS', 'drfp', 'dyfp', 'dcfp', 'a530', 'a480']
Making variable parameters figure
-  ['r', 'K', 'tlag', 'rc']
Making summary device figures
- Pcat-Pcat
- R100-S32
- R100-S34
- R33-S32
- R33-S34
- R33-S175
Making individual device figures
- Pcat-Pcat
- R100-S32
- R100-S34
- R33-S32
- R33-S34
- R33-S175
